I am gonne use Deep Q learning for this project.CNN would be a lot better and algorithms with N step lookahead algorithms seems to work better than that i will maybe try them after i learn more about monte carlo tree search.

Most of good algorithms combine policy learning(really similar to Q learning) and tree search algorithms.

https://jonathan-hui.medium.com/alphago-how-it-works-technically-26ddcc085319

neural network code i will use later =codelateruse.py



In [1]:
from kaggle_environments import make, evaluate, utils
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline
from gym import spaces
import random

<img src="./boardexample.png" alt="Drawing" style="width: 200px;"/>
obs.board property in this board will be

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 1, 2, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 2, 1, 2, 0, 2, 0]

I don't know where to start than i will copy another code with comments

In [2]:

class ConnectFourGym:

    def __init__(self, agent2="random"):
        ks_env = make('connectx', debug=True)
        self.env = ks_env.train([None, agent2])
        self.rows = ks_env.configuration.rows
        self.columns = ks_env.configuration.columns
        #We define actions possible here
        self.action_space = spaces.Discrete(self.columns)
        #We define all of space we are playing
        self.observation_space = spaces.Box(low=0,
                                            high=2,
                                            shape=(self.rows, self.columns, 1),
                                            dtype=int)
        # Tuple corresponding to the min and max possible rewards
        self.reward_range = (-10, 1)
        # StableBaselines throws error if these are not defined
        self.spec = None
        self.metadata = None

    def reset(self):
        self.obs = self.env.reset()
        return self.obs['board']

    def change_reward(self, old_reward, done):
        if old_reward == 1:
            return 1
        elif done:
            return -1
        else:
            return 0

    def step(self, agent):
        action = agent(self.obs)
        is_valid = (self.obs['board'][int(action)] == 0)
        if is_valid:
            self.obs, old_reward, done, _ = self.env.step(int(action))
            reward = self.change_reward(old_reward, done)
        else:
            reward, done, _ = -10, True, {}
        return [self.obs['board'],reward,done]

In [3]:
env = ConnectFourGym()

I am getting state action reward values with any agent

In [4]:
def get_values(agent,repeat_num=1000):
    i=0
    innerarr = []
    outerarr = []
    innerarr.append(env.reset())
    while i<repeat_num:
        new_state = env.step(agent)
        innerarr.append(new_state)
        if new_state[2]==True:
            outerarr.append(innerarr)
            innerarr = []
            i += 1
            innerarr.append(env.reset())
    return outerarr

Next i will use q learning 
- At first agent will move 100% random and it will decrease after every 100 game and agent will move according to neural network 1%(q_increase) more.It will stop at 97% (q_highest_prob).
- After every 1000 game we will update Q (repeat_num)
- Targets w will update after every 5(target_update_interval) updates.

<img src="./loss.png" alt="Drawing"/>

Lets define constants for changing if necessary

In [5]:
q_increase = 0.01
repeat_num = 1000
target_update_interval = 5
q_highest_prob = 0.97

In [6]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

model = Sequential(
    [Dense(64, activation='relu'),
     Dense(32, activation='relu'),
     Dense(7)])
optimizer = Adam(learning_rate=0.001)

In [7]:
def penalized_loss(reward):
  def custom_loss(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true) - K.square(y_true - reward), axis=-1)

  return custom_loss


In [ ]:
loss = custom_loss()